In [2]:
import time
import requests
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, RDFS, OWL, SKOS
from multiprocessing.dummy import Pool as ThreadPool

# Namespaces
schema = Namespace("https://schema.org/")
dbpedia = Namespace("http://dbpedia.org/resource/")
dbo = Namespace("http://dbpedia.org/ontology/")
ex = Namespace("http://example.org/")

# Cargar el grafo RDF original
input_file = "articles_base_data.ttl"
g = Graph()
g.parse(input_file, format="turtle")
print(f"Grafo cargado correctamente. Número de triples: {len(g)}")

# Extraer conceptos SKOS (máximo 20 para pruebas)
concepts = set()
for concept_uri in g.subjects(RDF.type, SKOS.Concept):
    for concept_label in g.objects(concept_uri, RDFS.label):
        concepts.add((concept_uri, str(concept_label).capitalize()))  # capitalize Case

print(f"Procesando {len(concepts)} conceptos.")

def format_label(label):
    """ Convierte el label a capitalize Case con formato DBpedia """
    return label.capitalize().replace(" ", "_")  # "social media" -> "Social_Media"

def enrich_concept(concept_data):
    """ Enlaza un concepto con DBpedia y agrega más información. """
    concept_uri, concept_label = concept_data
    formatted_label = format_label(concept_label)

    print(f"\nProcesando concepto: {concept_label} -> {formatted_label}")
    
    dbpedia_uri = get_dbpedia_concept_uri(formatted_label)
    if not dbpedia_uri:
        print(f"No se encontró URI de DBpedia para {concept_label}, probando SPARQL...")
        dbpedia_uri = search_dbpedia_sparql(concept_label)

    if not dbpedia_uri:
        print(f"No se encontró URI de DBpedia para {concept_label} después de SPARQL.")
        return None

    print(f"URI DBpedia encontrada: {dbpedia_uri}")
    
    enriched_triples = set()
    enriched_triples.add((concept_uri, OWL.sameAs, dbpedia_uri))

    # 🔥 **Consulta SPARQL mejorada** 🔥
    query = f"""
    SELECT ?prop ?value WHERE {{
        VALUES ?uri {{ <{dbpedia_uri}> }}
        VALUES ?prop {{ dbo:abstract rdfs:label }}
        ?uri ?prop ?value .
        FILTER(LANG(?value) = "en" || LANG(?value) = "es")
    }}
    """

    sparql_endpoint = "http://dbpedia.org/sparql"
    params = {"query": query, "format": "json"}

    try:
        response = requests.get(sparql_endpoint, params=params, timeout=5)
        if response.status_code == 200:
            data = response.json()
            print(f"Número de resultados encontrados: {len(data['results']['bindings'])}")
            
            for result in data["results"]["bindings"]:
                prop = URIRef(result["prop"]["value"])
                value = Literal(result["value"]["value"], lang=result["value"].get("xml:lang", ""))
                enriched_triples.add((dbpedia_uri, prop, value))
                print(f"✅ Agregado: {dbpedia_uri} {prop} {value}")
        else:
            print(f"❌ Error en la respuesta HTTP: {response.status_code}")
            print(f"Contenido de la respuesta: {response.text}")
            
    except Exception as e:
        print(f"❌ Error consultando DBpedia para {concept_label}: {e}")

    print(f"Total de triples enriquecidos para {concept_label}: {len(enriched_triples)}")
    return enriched_triples

def get_dbpedia_concept_uri(formatted_label):
    """ Intenta obtener la URI de un concepto en DBpedia directamente. """
    possible_uri = f"http://dbpedia.org/resource/{formatted_label}"
    return URIRef(possible_uri) if check_dbpedia_uri(possible_uri) else None

def check_dbpedia_uri(uri):
    """ Verifica si la URI de DBpedia existe """
    try:
        response = requests.head(uri, timeout=5)
        return response.status_code == 200
    except requests.RequestException:
        return False

def search_dbpedia_sparql(label):
    """ Busca una URI en DBpedia mediante SPARQL """
    query = f"""
    SELECT DISTINCT ?uri WHERE {{
        ?uri rdfs:label "{label}"@en .
        FILTER (STRSTARTS(STR(?uri), "http://dbpedia.org/resource/"))
    }} LIMIT 1
    """
    
    sparql_endpoint = "http://dbpedia.org/sparql"
    params = {"query": query, "format": "json"}

    try:
        response = requests.get(sparql_endpoint, params=params, timeout=5)
        if response.status_code == 200:
            data = response.json()
            results = data["results"]["bindings"]
            if results:
                uri = results[0]["uri"]["value"]
                print(f"Encontrada URI vía SPARQL: {uri}")
                return URIRef(uri)
            else:
                print(f"No se encontró URI en SPARQL para {label}")
    except Exception as e:
        print(f"Error en SPARQL para {label}: {e}")
    
    return None

# Procesar conceptos en paralelo
with ThreadPool(processes=4) as pool:
    results = pool.map(enrich_concept, concepts)

# Crear nuevo grafo enriquecido
enriched_graph = Graph()
enriched_graph.bind("dbo", dbo)
enriched_graph.bind("dbpedia", dbpedia)
enriched_graph.bind("schema", schema)
enriched_graph.bind("ex", ex)
enriched_graph.bind("owl", OWL)
enriched_graph.bind("skos", SKOS)

# Agregar tripletas obtenidas
for triples in results:
    if triples:
        for triple in triples:
            enriched_graph.add(triple)

print(f"Grafo enriquecido con {len(enriched_graph)} triples.")

# Guardar el grafo enriquecido
output_file = "pre_enriched_concepts1.ttl"
enriched_graph.serialize(output_file, format="turtle")
print(f"Grafo enriquecido guardado en {output_file}")


Grafo cargado correctamente. Número de triples: 98462
Procesando 3948 conceptos.

Procesando concepto: Topic analysis -> Topic_analysis

Procesando concepto: Continuous data -> Continuous_data

Procesando concepto: Trademark research data -> Trademark_research_data

Procesando concepto: Extremism analysis -> Extremism_analysis
No se encontró URI de DBpedia para Extremism analysis, probando SPARQL...
No se encontró URI de DBpedia para Trademark research data, probando SPARQL...
No se encontró URI de DBpedia para Continuous data, probando SPARQL...
No se encontró URI de DBpedia para Topic analysis, probando SPARQL...
No se encontró URI en SPARQL para Extremism analysis
No se encontró URI de DBpedia para Extremism analysis después de SPARQL.

Procesando concepto: Ml methods -> Ml_methods
Encontrada URI vía SPARQL: http://dbpedia.org/resource/Continuous_data
URI DBpedia encontrada: http://dbpedia.org/resource/Continuous_data
No se encontró URI en SPARQL para Trademark research data
No se e

#### CORRECION DE URI

In [3]:
import re

# Ruta del archivo original y del archivo corregido
input_file = "pre_enriched_concepts1.ttl"
output_file = "enriched_concepts1.ttl"

# Expresión regular para encontrar y reemplazar en URIs específicas
pattern = re.compile(r"<http://example.org/concept/(.*?)>")

with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf-8") as outfile:
    for line in infile:
        modified_line = pattern.sub(r"<http://example.org/sa/\1>", line)
        outfile.write(modified_line)

print("Archivo corregido guardado como", output_file)


Archivo corregido guardado como enriched_concepts1.ttl


In [4]:
!jupyter nbconvert --to html enriquecimientoKeywords.ipynb

[NbConvertApp] Converting notebook enriquecimientoKeywords.ipynb to html
[NbConvertApp] Writing 3292837 bytes to enriquecimientoKeywords.html
